In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from skimage.feature import local_binary_pattern
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report

Initial image

In [2]:
# Create a data frame to store the dataset and use number to represent a class
def initImg(root, matrix_label):
    label_lst, class_lst, img_lst = [], [], []
    label = 0
    for file in os.listdir(root):
        class_path = os.path.join(root, file)
        for fn in os.listdir(class_path):
            img_path = os.path.join(class_path, fn)
            img_lst.append(img_path)
            class_lst.append(file)
            if file not in matrix_label:
                matrix_label.append(file)
            label_lst.append(label)
        label += 1

    return pd.DataFrame({
        "filename": img_lst,
        "class": class_lst,
        "label": label_lst
    })

Feature Extraction Utilities

- SIFT

In [3]:
# preprocess the dataset and get descriptors for each image and build histogram
def preprocessImg(img):
    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    laplacian = cv2.convertScaleAbs(laplacian)
    sharpened = cv2.addWeighted(img, 1, laplacian, 1.0, 0)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(sharpened)
    return clahe_img


def build_bow_histogram(descriptors, kmeans, n_clusters):
    if descriptors is None:
        return np.zeros(n_clusters)

    descriptors = np.asarray(descriptors, dtype=np.float32)
    words = kmeans.predict(descriptors)

    histogram, _ = np.histogram(words, bins=np.arange(n_clusters + 1))

    return histogram


def get_descriptors(img_path, preprocess=True):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if preprocess:
        gray = preprocessImg(gray)

    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(gray, None)

    return descriptors


def descriptorlst(train_df):
    descriptor_lst = []

    for _, row in train_df.iterrows():
        descriptors = get_descriptors(row['filename'], preprocess=True)
        if descriptors is not None:
            descriptor_lst.extend(descriptors)
    descriptor_lst = np.asarray(descriptor_lst, dtype=np.float32)

    return descriptor_lst

- LBP

In [4]:
radius = 1
n_points = 8 * radius
method = 'uniform'

def extract_lbp_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, n_points, radius, method).astype(np.uint8)

    hist = cv2.calcHist([lbp], [0], None, [n_points + 2], [0, n_points + 2])
    hist = cv2.normalize(hist, hist).flatten()

    return hist

Dataloader

In [5]:
def dataloader(data, preprocess):
    train_df, test_df = train_test_split(
        data,
        test_size=0.2,
        random_state=42,
        stratify=data['label']
    )
    
    X_train, y_train, X_test, y_test = [], [], [], []
    if preprocess == 'sift':
        descriptor_lst = descriptorlst(train_df)
        kmeans = MiniBatchKMeans(n_clusters=100, random_state=42)
        kmeans.fit(descriptor_lst)
        for df, X, y in [(train_df, X_train, y_train), (test_df, X_test, y_test)]:
            for _, row in df.iterrows():
                descriptors = get_descriptors(row['filename'], preprocess=False)
                hist = build_bow_histogram(descriptors, kmeans, 100)
                X.append(hist)
                y.append(row['label'])

    elif preprocess == 'lbp':
         for df, X, y in [(train_df, X_train, y_train), (test_df, X_test, y_test)]:
            for _, row in df.iterrows():
                features = extract_lbp_features(row['filename'])
                X.append(features)
                y.append(row['label'])

    return np.array(X_train), y_train, np.array(X_test), y_test

Model Training and Evaluation

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, average='macro'))
    print("Recall:", recall_score(y_test, y_pred, average='macro'))
    print("F1 Score:", f1_score(y_test, y_pred, average='macro'))

    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=matrix_lst, yticklabels=matrix_lst)
    plt.show()

In [10]:
def run_models(X_train, y_train, X_test, y_test, descriptor_name):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=500),
        'KNN': KNeighborsClassifier(n_neighbors=5)
    }

    print(f"\n=== Results for {descriptor_name.upper()} Features ===")
    for model_name, model in models.items():
        print(f"\n-- {model_name} --")
        model.fit(X_train, y_train)
        evaluate_model(model, X_test, y_test)


# Initialize dataset
root_dir = 'Aerial_Landscapes'
matrix_label = []
df = initImg(root_dir, matrix_label)

# Run for both SIFT and LBP
for descriptor_type in ['sift', 'lbp']:
    X_train, y_train, X_test, y_test = dataloader(df, preprocess=descriptor_type)
    run_models(X_train, y_train, X_test, y_test, descriptor_type)



=== Results for SIFT Features ===

-- Logistic Regression --


c:\Users\LeeX0\.conda\envs\COMP9517\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6241666666666666
Precision: 0.6256437588083007
Recall: 0.6241666666666669
F1 Score: 0.6222975678636975

Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.60      0.59       160
           1       0.54      0.50      0.52       160
           2       0.65      0.61      0.63       160
           3       0.69      0.64      0.66       160
           4       0.46      0.56      0.51       160
           5       0.84      0.74      0.79       160
           6       0.47      0.65      0.55       160
           7       0.59      0.59      0.59       160
           8       0.36      0.28      0.31       160
           9       0.75      0.72      0.73       160
          10       0.87      0.90      0.88       160
          11       0.71      0.64      0.68       160
          12       0.68      0.69      0.69       160
          13       0.72      0.84      0.78       160
          14       0.47      0.40      0.43      